## import latest email bounces

In [1]:
import numpy as np
import pandas as pd
import os
from DB2SalesforceAPI import DB2SalesforceAPI

sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",
}

In [2]:
cwd = os.getcwd()

In [3]:
# import the dataset
filename = cwd+'/SF email bounces/Email Bounces For Travis20210303.csv'#'JourneyEmailBounces20201112.csv'
bounce_df = pd.read_csv(filename)
display(bounce_df.head(5))

,SendTime,BounceTime,EmailAddress,ContactKey,ContactID,ListID,BatchID,JobID
0,2/5/2021 1:30:47 PM,2/5/2021 1:57:23 PM,fparnold@chem.northwestern.edu,0035w000034JHbMAAW,135097783,221,5,362106
1,2/5/2021 1:30:47 PM,2/5/2021 1:36:03 PM,mjsteven@purdue.edu,0035w000034JHboAAG,135097786,221,5,362106
2,2/5/2021 1:30:47 PM,2/5/2021 1:30:59 PM,erehder@kopin.com,0035w000034JHbPAAW,135097789,221,5,362106
3,2/5/2021 1:30:47 PM,2/5/2021 1:36:04 PM,kapooro@purdue.edu,0035w000034JHcJAAW,135097839,221,5,362106
4,2/5/2021 1:30:47 PM,2/5/2021 1:36:00 PM,tewers@berkeley.edu,0035w000034JHclAAG,135097842,221,5,362106


In [4]:
sparse_bdf = bounce_df[['EmailAddress','ContactKey']].rename(columns={'EmailAddress':'Email','ContactKey':'Id'}).drop_duplicates()
display(sparse_bdf.head(5))
print(sparse_bdf.shape)

,Email,Id
0,fparnold@chem.northwestern.edu,0035w000034JHbMAAW
1,mjsteven@purdue.edu,0035w000034JHboAAG
2,erehder@kopin.com,0035w000034JHbPAAW
3,kapooro@purdue.edu,0035w000034JHcJAAW
4,tewers@berkeley.edu,0035w000034JHclAAG


(39538, 2)


## Import SF contact data

In [5]:
## since the data already has contactkey, we can use that as ID and should be good
sparse_bdf['Email_Bounced__c'] = True
display(sparse_bdf.head(5))

,Email,Id,Email_Bounced__c
0,fparnold@chem.northwestern.edu,0035w000034JHbMAAW,True
1,mjsteven@purdue.edu,0035w000034JHboAAG,True
2,erehder@kopin.com,0035w000034JHbPAAW,True
3,kapooro@purdue.edu,0035w000034JHcJAAW,True
4,tewers@berkeley.edu,0035w000034JHclAAG,True


## Begin upsert to SF

In [6]:
# api object
db_s = DB2SalesforceAPI(sf_login_params)

# send data to SF
db_s.object_id = 'Contact'
db_s.external_id = 'Id'

db_s.send_data(sparse_bdf)

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000UFXL9AAP
[Success] CSV upload successful. Job ID = 7505w00000UFXL9AAP
[Success] Closing job successful. Job ID = 7505w00000UFXL9AAP


In [11]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000UFXL9AAP',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-03-04T16:06:14.000+0000',
 'systemModstamp': '2021-03-04T16:08:30.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 39538,
 'numberRecordsFailed': 29,
 'retries': 0,
 'totalProcessingTime': 474146,
 'apiActiveProcessingTime': 430469,
 'apexProcessingTime': 3458021}

In [12]:
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",Email,Email_Bounced__c,Id\n'
 '"","INVALID_ID_FIELD:\'00Q5w00001tLZdF\' is not a valid Salesforce ID for '
 'the type '
 'Contact:--","inigo.anza@constellium.com","true","00Q5w00001tLZdFEAW"\n'
 '"","INVALID_ID_FIELD:\'00Q5w00001uV9d0\' is not a valid Salesforce ID for '
 'the type Contact:--","dmcilroy@uidaho.edu","true","00Q5w00001uV9d0EAC"\n'
 '"","INVALID_ID_FIELD:\'00Q5w00001uV9hG\' is not a valid Salesforce ID for '
 'the type '
 'Contact:--","kratzer@fhi-berlin.mpg.de","true","00Q5w00001uV9hGEAS"\n'
 '"","INVALID_ID_FIELD:\'00Q5w00001uV9hL\' is not a valid Salesforce ID for '
 'the type '
 'Contact:--","tarucha@phys.s.u-tokyo.ac.jp","true","00Q5w00001uV9hLEAS"\n'
 '"","INVALID_ID_FIELD:\'00Q5w00001uV9d5\' is not a valid Salesforce ID for '
 'the type Contact:--","fzahid@hku.hk","true","00Q5w00001uV9d5EAC"\n'
 '"","INVALID_ID_FIELD:\'00Q5w00001uV9dK\' is not a valid Salesforce ID for '
 'the type '
 'Contact:--","romtabe@rie.shizuoka.ac.jp","true","00Q5w000

## lead email bounces

In [26]:
#import all leads
db_l = DB2SalesforceAPI(sf_login_params)

sf_df = db_l.query_data("select id, Email__c from Lead")

Obtained Salesforce access token ...... True
[Success] Bulk job creation successful. Job ID = 7505w00000UFXlXAAX
{"id":"7505w00000UFXlXAAX","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT","createdDate":"2021-03-04T16:29:42.000+0000","systemModstamp":"2021-03-04T16:29:42.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000UFXlXAAX","operation":"query","object":"Lead","createdById":"0055w00000DM5bOAAT","createdDate":"2021-03-04T16:29:42.000+0000","systemModstamp":"2021-03-04T16:29:47.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":7364,"retries":0,"totalProcessingTime":1130}
[Success] Bulk job completed successfully.


In [27]:
display(sf_df.head(5))

,Email__c,Id
0,NaN,00Q5w00001otOM1EAM
1,NaN,00Q5w00001otOM2EAM
2,NaN,00Q5w00001otOM3EAM
3,NaN,00Q5w00001otOM4EAM
4,NaN,00Q5w00001otOM5EAM


In [43]:
import numpy as np
temp = np.where(np.in1d(sparse_bdf['Id'].to_numpy(),sf_df['Id'].to_numpy()) == True)

In [45]:
lead_ids = sparse_bdf['Id'].to_numpy()[temp]

29

In [47]:
lead_df = pd.DataFrame()
lead_df['Id'] = lead_ids
lead_df['Email_Bounced__c'] = True

In [49]:
display(lead_df.head(5))

,Id,Email_Bounced__c
0,00Q5w00001uV9dGEAS,True
1,00Q5w00001uudb5EAA,True
2,00Q5w00001tLZdFEAW,True
3,00Q5w00001uV9d0EAC,True
4,00Q5w00001uV9hGEAS,True


In [50]:
db_l = DB2SalesforceAPI(sf_login_params)

db_l.object_id = 'Lead'
db_l.external_id = 'Id'

Obtained Salesforce access token ...... True


In [51]:
db_l.send_data(lead_df)

[Success] Bulk job creation successful. Job ID = 7505w00000UFXy6AAH
[Success] CSV upload successful. Job ID = 7505w00000UFXy6AAH
[Success] Closing job successful. Job ID = 7505w00000UFXy6AAH


In [54]:
# check status
db_l.check_bulk_status()

{'id': '7505w00000UFXy6AAH',
 'operation': 'upsert',
 'object': 'Lead',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-03-04T16:38:55.000+0000',
 'systemModstamp': '2021-03-04T16:39:01.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 29,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 315,
 'apiActiveProcessingTime': 229,
 'apexProcessingTime': 58}